Готовый код, в котором надо будет поменять модель (используется библиотека `transformers`): 
    
https://colab.research.google.com/drive/1iisf0y77wtNFCdnqYZOgkQfxtz53Rzqa?usp=sharing

In [1]:
# pip install wikidata

In [2]:
TRAIN_DATA_PATH = './QALD_10/data/qald_9_plus/qald_9_plus_train_wikidata.json'
TEST_DATA_PATH = './QALD_10/data/qald_10/qald_10.json'

In [3]:
import pandas as pd

In [25]:
df_train_raw = pd.read_json(TRAIN_DATA_PATH, orient='index').T

In [5]:
def get_questions_answers(s):
    for_df_list = []
    for q_lang in s['question']:
        q_lang['question_id'] = s['id']
        
        key = list(s['answers'][0]['results']['bindings'][0].keys())[0]
        q_lang['answer_requests'] = [
            el[key]['value'].replace('http://www.wikidata.org/entity/', '') for el in s['answers'][0]['results']['bindings']
        ]
        for_df_list.append(q_lang)
        
    return for_df_list

In [6]:
train_list = []
for i in range(len(df_train_raw)):
    train_list.extend(
        get_questions_answers(df_train_raw.iloc[i].values[0])
    )
df_train = pd.DataFrame(train_list)
df_train

,language,string,question_id,answer_requests
0,en,What is the time zone of Salt Lake City?,1,"[Q2212, Q3134980]"
1,de,In welcher Zeitzone liegt Salt Lake City?,1,"[Q2212, Q3134980]"
2,de,Was ist die Zeitzone von Salt Lake City?,1,"[Q2212, Q3134980]"
3,ru,Какой часовой пояс в Солт-Лейк-Сити,1,"[Q2212, Q3134980]"
4,ru,В каком часовом поясе расположен Солт-Лейк-Сити?,1,"[Q2212, Q3134980]"
...,...,...,...,...
4058,ru,Кто открыл Плутон,412,[Q190232]
4059,uk,Хто відкрив Плутон?,412,[Q190232]
4060,lt,Kas atrado Plutoną?,412,[Q190232]
4061,be,Хто адкрыў Плутон,412,[Q190232]


In [36]:
df_train['question_id'][df_train['language'] == 'en'].nunique()

412

In [37]:
df_train['question_id'].nunique()

412

Каждый вопрос задан на нескольких языках, причём для каждого вопроса имеется его версия на английском языке $\implies$ можно использовать датасет для английского языка (и, соответственно, только для него делать предсказания) $\implies$ можно взять какой-нибудь известный датасет (например, SQuAD (Stanford Question Answering Dataset)) и построить для него какую-то модель (возьмем что-то из SOTA: BERT or GPT)  

Models for question answering tasks in`transformers` library (all of them are also available in `simple transformers` https://simpletransformers.ai/docs/qa-specifics/#supported-model-types): 
* BERT
* XLM
* XLNet
* RoBERTa 
* DistilBERT
* CamamBERT (seems like it is for French)
* AlBERT
* XLMRoBERTa
* FlauBERT (seems like it is for French)
* Bart
* ELECTRA
* Reformer
* Longformer
* MobileBERT

In table 4 RoBERTa seems the best (among BERT, DistilBERT, RoBERTa)
https://docs.yandex.ru/docs/view?tm=1652195442&tld=ru&lang=en&name=2111.08546.pdf&text=squad%20bert%20roberta%20comparison&url=https%3A%2F%2Farxiv.org%2Fpdf%2F2111.08546.pdf&lr=21745&mime=pdf&l10n=ru&sign=3528f8a5d40db7011d2ef05bcf003897&keyno=0&serpParams=tm%3D1652195442%26tld%3Dru%26lang%3Den%26name%3D2111.08546.pdf%26text%3Dsquad%2Bbert%2Broberta%2Bcomparison%26url%3Dhttps%253A%2F%2Farxiv.org%2Fpdf%2F2111.08546.pdf%26lr%3D21745%26mime%3Dpdf%26l10n%3Dru%26sign%3D3528f8a5d40db7011d2ef05bcf003897%26keyno%3D0

# -------------------------------------------------------------------------------------------------------

In [7]:
from wikidata.client import Client
client = Client()  # doctest: +SKIP
entity = client.get('Q190232', load=True)
print(entity)
print(entity.description)

<wikidata.entity.Entity Q190232 'Clyde Tombaugh'>
American astronomer (1906-1997)


In [9]:
entity.data.keys()

dict_keys(['pageid', 'ns', 'title', 'lastrevid', 'modified', 'type', 'id', 'labels', 'descriptions', 'aliases', 'claims', 'sitelinks'])

In [19]:
entity.data['modified']

'2022-02-24T13:17:30Z'

In [12]:
entity.data['labels']['ru']['value'], entity.data['descriptions']['ru']['value']

('Томбо, Клайд Уильям', 'американский астроном')

In [68]:
df_train['answer'] = df_train['answer_requests'].apply(lambda x: [client.get(el, load=True).description for el in x])

KeyboardInterrupt: 

In [ ]:
df_train

In [39]:
pip install deeppavlov

     |████████████████████████████████| 878 kB 1.4 MB/s eta 0:00:01
     |████████████████████████████████| 2.9 MB 29.2 MB/s eta 0:00:01
     |████████████████████████████████| 65 kB 3.4 MB/s  eta 0:00:01
     |████████████████████████████████| 2.1 MB 36.1 MB/s eta 0:00:01
     |████████████████████████████████| 510 kB 33.2 MB/s eta 0:00:01
     |████████████████████████████████| 46 kB 3.9 MB/s  eta 0:00:01
     |████████████████████████████████| 8.2 MB 26.8 MB/s eta 0:00:01
     |████████████████████████████████| 43 kB 1.6 MB/s  eta 0:00:01
     |████████████████████████████████| 26.1 MB 34.7 MB/s eta 0:00:01
     |████████████████████████████████| 20.1 MB 33.1 MB/s eta 0:00:01
     |████████████████████████████████| 10.4 MB 29.7 MB/s eta 0:00:01
     |████████████████████████████████| 57 kB 4.3 MB/s  eta 0:00:01
     |████████████████████████████████| 55 kB 3.2 MB/s  eta 0:00:01
     |████████████████████████████████| 82 kB 1.0 MB/s  eta 0:00:01
     |████████████████████████████████

In [ ]:
pip install tensorflow

In [40]:
from deeppavlov import build_model, configs

model = build_model(configs.squad.squad, download=True)
model(['DeepPavlov is library for NLP and dialog systems.'], ['What is DeepPavlov?'])

2022-05-10 13:07:08.790 INFO in 'deeppavlov.core.data.utils'['utils'] at line 95: Downloading from http://files.deeppavlov.ai/embeddings/wiki-news-300d-1M-char.vec to /root/.deeppavlov/downloads/embeddings/wiki-news-300d-1M-char.vec
100%|██████████| 7.73M/7.73M [00:00<00:00, 15.1MB/s]
2022-05-10 13:07:10.332 INFO in 'deeppavlov.core.data.utils'['utils'] at line 95: Downloading from http://files.deeppavlov.ai/embeddings/wiki-news-300d-1M.vec to /root/.deeppavlov/downloads/embeddings/wiki-news-300d-1M.vec
100%|██████████| 2.26G/2.26G [00:45<00:00, 49.5MB/s] 
2022-05-10 13:07:57.0 INFO in 'deeppavlov.core.data.utils'['utils'] at line 95: Downloading from http://files.deeppavlov.ai/deeppavlov_data/squad_model_1.4_cpu_compatible.tar.gz to /root/.deeppavlov/squad_model_1.4_cpu_compatible.tar.gz
100%|██████████| 222M/222M [00:04<00:00, 45.0MB/s] 
2022-05-10 13:08:02.475 INFO in 'deeppavlov.core.data.utils'['utils'] at line 272: Extracting /root/.deeppavlov/squad_model_1.4_cpu_compatible.tar.g

ModuleNotFoundError: No module named 'tensorflow'

In [42]:
df_train['string'][df_train['language'] == 'en'].values

array(['What is the time zone of Salt Lake City?', 'Who killed Caesar?',
       'What is the highest mountain in Germany?',
       'Which artists were born on the same date as Rachel Stevens?',
       'What is the profession of Frank Herbert?',
       'How many seats does the home stadium of FC Porto have?',
       'Which European countries have a constitutional monarchy?',
       'Which countries have places with more than two caves?',
       'What is Angela Merkel’s birth name?',
       'Who is the mayor of Berlin?',
       'Which countries in the European Union adopted the Euro?',
       'Which country was Bill Gates born in?',
       'How many grand-children did Jacques Cousteau have?',
       'Give me all professional skateboarders from Sweden.',
       'Which monarchs of the United Kingdom were married to a German?',
       'Give me all Argentine films.', 'How did Michael Jackson die?',
       'Which U.S. state has been admitted latest?',
       'Which classes does the Millepede 

In [25]:
question = df_train_raw.iloc[0].values[0]
question

{'id': '1',
 'question': [{'language': 'en',
   'string': 'What is the time zone of Salt Lake City?',
   'question_id': '1'},
  {'language': 'de', 'string': 'In welcher Zeitzone liegt Salt Lake City?'},
  {'language': 'de', 'string': 'Was ist die Zeitzone von Salt Lake City?'},
  {'language': 'ru', 'string': 'Какой часовой пояс в Солт-Лейк-Сити'},
  {'language': 'ru',
   'string': 'В каком часовом поясе расположен Солт-Лейк-Сити?'},
  {'language': 'uk', 'string': 'Який часовий пояс у Солт-Лейк Сіті?'},
  {'language': 'lt', 'string': 'Kokia Solt Leik Sičio laiko zona?'},
  {'language': 'be', 'string': 'Які гадзінны пояс у Солт-Лэйк-Сіці'},
  {'language': 'lt', 'string': 'Kokia laiko juosta yra Solt Leik Sityjes'},
  {'language': 'ba', 'string': 'Ниндей вакыт поясы Солт-Лейк-Ситила'}],
 'query': {'sparql': 'SELECT DISTINCT ?o1 WHERE { <http://www.wikidata.org/entity/Q23337>  <http://www.wikidata.org/prop/direct/P421>  ?o1 .  }'},
 'answers': [{'head': {'vars': ['o1']},
   'results': {'bi

In [22]:
train_list = []
for q_lang in question['question']:
    q_lang['question_id'] = question['id']
    train_list.append(q_lang)
pd.DataFrame(train_list)

,language,string,question_id
0,en,Which artists were born on the same date as Ra...,4
1,de,Welche Künstler sind am gleichen Tag wie Rache...,4
2,ru,Какие актеры родились в тоже день с Рэйчел Сти...,4
3,ru,Какие художники родились в один день с Рэйчел ...,4
4,ru,"Какие артисты родились в тот же день, что и Ре...",4
5,ru,Кто из артистов родился в один день с Рэйчел С...,4
6,uk,Які актори народились в один день з Рейчел Сті...,4
7,uk,"Які художники народилися того ж самого дня, що...",4
8,be,Якія акцёры нарадзіліся ў адзiн дзень з Рэйчэл...,4
9,lt,Kurie aktoriai gimė tą pačią dieną kaip Rachel...,4


In [41]:
question['answers'][0]['results']['bindings'][0][list(question['answers'][0]['results']['bindings'][0].keys())[0]]['value']

'http://www.wikidata.org/entity/Q2212'

In [29]:
[el['s']['value'].replace('http://www.wikidata.org/entity/', '') for el in question['answers'][0]['results']['bindings']]

KeyError: 's'

In [32]:
for k, v in question.items():
    print(k)
    print(v)
    print()

id
2

question
[{'language': 'en', 'string': 'Who killed Caesar?'}, {'language': 'de', 'string': 'Wer hat Caesar ermordet?'}, {'language': 'de', 'string': 'Wer hat Cäsar getötet?'}, {'language': 'ru', 'string': 'Кто убил Цезаря'}, {'language': 'uk', 'string': 'Хто вбив Цезаря?'}, {'language': 'lt', 'string': 'Kas nužudė Cezarį?'}, {'language': 'be', 'string': 'Хто забіў Цэзара'}, {'language': 'ba', 'string': 'Кем Цезарьзы ултергэн?'}]

query
{'sparql': 'SELECT DISTINCT ?o1 WHERE { <http://www.wikidata.org/entity/Q1048>  <http://www.wikidata.org/prop/direct/P157>  ?o1 .  }'}

answers
[{'head': {'vars': ['o1']}, 'results': {'bindings': [{'o1': {'type': 'uri', 'value': 'http://www.wikidata.org/entity/Q1228715'}}, {'o1': {'type': 'uri', 'value': 'http://www.wikidata.org/entity/Q1243545'}}, {'o1': {'type': 'uri', 'value': 'http://www.wikidata.org/entity/Q172248'}}, {'o1': {'type': 'uri', 'value': 'http://www.wikidata.org/entity/Q207370'}}, {'o1': {'type': 'uri', 'value': 'http://www.wikidat

In [26]:
pd.DataFrame(df_train.iloc[0].values[0])

ValueError: arrays must all be same length